In [1]:
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from collections import Counter, defaultdict
import community.community_louvain as community_louvain
import numpy as np
import re

In [8]:
dir_path = "C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/net_movies_info"
csv = "C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/csv/unified_dictionary_percent.csv"
df = pd.read_csv(csv)

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Leer el CSV
# df = pd.read_csv("path_to_your_file.csv")

# Crear un diccionario para almacenar las palabras por género
genre_keywords = {genre: [] for genre in df.columns[1:]}  # Ignoramos la primera columna de palabras

# Clasificar las palabras por género
for _, row in df.iterrows():
    word = row[0]  # Primera columna es la palabra clave
    for genre, value in row[1:].items():  # Iteramos sobre los géneros
        if value >= 1.0:
            genre_keywords[genre].append(word)

# Ahora calculamos el TF-IDF para cada lista de palabras por género
tfidf_results = {}

for genre, words in genre_keywords.items():
    # Crear un TF-IDF vectorizer y calcular el TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(words)])  # Unimos las palabras en una sola cadena
    tfidf_results[genre] = dict(zip(vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0]))

# Mostrar los resultados (TF-IDF para cada palabra en cada género)
# for genre, tfidf_scores in tfidf_results.items():
#     print(f"TF-IDF scores for {genre}:")
#     for word, score in sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True):
#         print(f"{word}: {score}")


C:\Users\nerea\AppData\Local\Temp\ipykernel_16528\1933202204.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  word = row[0]  # Primera columna es la palabra clave


In [10]:
def extract_synopsis(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Use a regular expression to extract the synopsis (assuming it's between "Synopsis:" and "]")
    match = re.search(r'Synopsis:\s*\[(.*?)\]', content, re.DOTALL)
    if match:
        synopsis = match.group(1)
        return synopsis
    else:
        return ""

# Get a list of all files in the directory
files = os.listdir(dir_path)

# Extract synopses from each file
synopses = {}
for file in files:
    file_path = os.path.join(dir_path, file)
    if os.path.isfile(file_path):
        synopsis = extract_synopsis(file_path)
        # You can associate the synopsis with the movie name, here we use the file name as the movie name
        movie_name = os.path.splitext(file)[0]
        synopses[movie_name] = synopsis

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Clean all synopses
for movie, synopsis in synopses.items():
    synopses[movie] = clean_text(synopsis)

In [11]:
from collections import Counter

def calculate_genre_percentages(synopsis, keyword_genre_mapping, tfidf_scores):
    # Clean the synopsis and split into words
    cleaned_synopsis = clean_text(synopsis)
    words = cleaned_synopsis.split()  # All words in the synopsis (not unique)
    
    genre_contributions = Counter()  # To store the weighted genre contributions
    total_keywords = 0  # To store the total number of genre-related keywords found
    genre_percentages = []

    # For each word in the synopsis
    for word in words:
        if word in keyword_genre_mapping:  # If the word is in the mapping
            for genre in keyword_genre_mapping[word]:
                if word in tfidf_scores[genre]:  # Check if the word has a TF-IDF score for that genre
                    genre_contributions[genre] += tfidf_scores[genre].get(word, 0)  # Add TF-IDF score

    # Generate percentages
    total_contribution = sum(genre_contributions.values())
    if total_contribution > 0:  # To avoid division by zero
        for genre, contribution in genre_contributions.items():
            genre_percentages.append((genre, (contribution / total_contribution) * 100))

    return genre_percentages


In [13]:
keyword_genre_mapping = df.set_index('Keyword').to_dict(orient='index')

movie_genre_percentages = {}

for movie_name, synopsis in synopses.items():
    genre_percentages = calculate_genre_percentages(synopsis, keyword_genre_mapping, tfidf_results)
    movie_genre_percentages[movie_name] = genre_percentages

In [14]:
def save_csv_percentages(movie_genre_percentages, output_file):
    data = []
    for movie_name, genre_percentages in movie_genre_percentages.items():
        # Sort genres by percentage in descending order
        sorted_genres = sorted(genre_percentages, key=lambda x: x[1], reverse=True)
        
        # Prepare data for saving
        data.append({
            "Movie": movie_name,
            "Top Genre": sorted_genres[0][0] if sorted_genres else None,  # Highest percentage genre
            "Top Percentage": sorted_genres[0][1] if sorted_genres else 0,  # Highest percentage
            "All Genres": ", ".join([f"{genre}: {percentage:.2f}%" for genre, percentage in sorted_genres])  # Full list
        })
    
    # Convert to DataFrame and save
    result_df = pd.DataFrame(data)
    result_df.to_csv(output_file, index=False)
    print(f"Genre percentages saved to {output_file}")

In [15]:
output_file = "movie_genre_tfIDF_top_prueba.csv"
save_csv_percentages(movie_genre_percentages, output_file)

Genre percentages saved to movie_genre_tfIDF_top_prueba.csv


Bottom Movies

In [19]:
dir_path = "C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/bottom_net_movies_info"
csv = "C:/Users/nerea/Documents/MasterDTU/SocialGraphs_fall24/Projects/socialGraphs_fall24/FinalProject/csv/unified_dictionary_percent.csv"
df = pd.read_csv(csv)

# Get a list of all files in the directory
files = os.listdir(dir_path)

# Crear un diccionario para almacenar las palabras por género
genre_keywords = {genre: [] for genre in df.columns[1:]}  # Ignoramos la primera columna de palabras

# Clasificar las palabras por género
for _, row in df.iterrows():
    word = row[0]  # Primera columna es la palabra clave
    for genre, value in row[1:].items():  # Iteramos sobre los géneros
        if value >= 1.0:
            genre_keywords[genre].append(word)

# Ahora calculamos el TF-IDF para cada lista de palabras por género
tfidf_results = {}

for genre, words in genre_keywords.items():
    # Crear un TF-IDF vectorizer y calcular el TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(words)])  # Unimos las palabras en una sola cadena
    tfidf_results[genre] = dict(zip(vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0]))

# Extract synopses from each file
synopses = {}
for file in files:
    file_path = os.path.join(dir_path, file)
    if os.path.isfile(file_path):
        synopsis = extract_synopsis(file_path)
        # You can associate the synopsis with the movie name, here we use the file name as the movie name
        movie_name = os.path.splitext(file)[0]
        synopses[movie_name] = synopsis

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Clean all synopses
for movie, synopsis in synopses.items():
    synopses[movie] = clean_text(synopsis)

# Convert the DataFrame into a dictionary for quick lookups
keyword_genre_mapping = df.set_index('Keyword').to_dict(orient='index')

movie_genre_percentages = {}

for movie_name, synopsis in synopses.items():
    genre_percentages = calculate_genre_percentages(synopsis, keyword_genre_mapping, tfidf_results)
    movie_genre_percentages[movie_name] = genre_percentages

output_file = "movie_genre_tfIDF_bottom_prueba.csv"
save_csv_percentages(movie_genre_percentages, output_file)

C:\Users\nerea\AppData\Local\Temp\ipykernel_16528\599209862.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  word = row[0]  # Primera columna es la palabra clave


Genre percentages saved to movie_genre_tfIDF_bottom_prueba.csv
